# Input

This setup capture the problem with specific constraints on node sums and paths.

1. Node Sum Constraints
A dictionary where each key represents a node, and its value is the sum constraint for all edges directly connected to this node.

2. Node Path Constraints
A dictionary where each key represents a node, and its value is a list of possible path sum constraints starting from that node.

3. Edges with Weights
A list of tuples representing edges in the graph. Each tuple contains two vertices and an edge weight. If the weight is unknown, it is represented by an alphabetic character.

4. Search Space
A list of allowed integer values that can be used to replace the unknown edge weights.

5. Number of Vertices
The total number of vertices in the graph.

In [254]:
node_sum_constraints = {
    0: 17, 
    1: 3,
    7: 54,
    8: 49,
    9: 60,
    10: 79,
    11: 75,
    12: 29,
    15: 39,
    16: 25
}

node_path_constraints = {
    2: [31],
    3: [19, 23],
    5: [8],
    6: [6, 9, 16]
}

edges_with_weights = [
    (0, 1, 'b'), (0, 3, 'a'), (1, 4, 'c'), (3, 4, 12),
    (3, 7, 'e'), (2, 7, 'd'), (6, 9, 'j'), (4, 8, 'f'),
    (5, 8, 'g'), (7, 9, 'i'), (9, 8, 'k'), (7, 10, 'h'),
    (9, 10, 24), (9, 11, 'n'), (10, 13, 'l'), (11, 14, 'p'),
    (10, 12, 'm'), (10, 15, 7), (11, 16, 'o'), (12, 15, 'q'),
    (8, 11, 20), (12, 16, 'r'), (15, 17, 's'), (16, 17, 't'),
]

search_space = list(range(1,25))
num_vertices = 18

# Preparation

The goal is to prepare a graph structure with vertices and edges, handle known and unknown edge weights, and set up equations representing node sum constraints. The provided Graph class will be used to create and manage the graph. We will also prepare lists of allowed integers and equations for the constraint solver.

Steps
- Define the Graph Class
- Initialize the Graph with Vertices and Edges
- Prepare the Disallowed List and Variable Names
- Create Allowed Integers List


In [255]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.edges = [[-1 for _ in range(vertices)] for _ in range(vertices)]
    
    def add_edge(self, u, v, weight):
        self.edges[u][v] = weight
        self.edges[v][u] = weight
    
    def get_edge(self, u, v):
        return self.edges[u][v]
    
    def neighbors(self, node):
        return [i for i, weight in enumerate(self.edges[node]) if weight != -1]

graph = Graph(num_vertices) 

for u, v, weight in edges_with_weights:
    graph.add_edge(u, v, weight)

disallowed_list = []
variable_names = []

for _, _, weight in edges_with_weights:
    if isinstance(weight, int):
        disallowed_list.append(weight)
    else:
        variable_names.append(weight)

allowed_integers = [i for i in search_space if i not in disallowed_list]
equations = ["+".join(str(graph.get_edge(node, n)) for n in graph.neighbors(node))+f'={target_sum}' for node, target_sum in node_sum_constraints.items()]

# Sum constraints

This is the process of using a Z3 solver to find solutions that fulfill the sum constraints for the graph problem. The process involves setting up the solver, defining variables, applying constraints, and extracting solutions.

Step-by-Step Guide
- Import Z3 Solver
- Initialize Variables
- Setup Solver
- Apply Distinct Constraint
- Apply Allowed Integers Constraint
- Create Variable Map
- Parse and Apply Sum Constraints
- Find All Solutions

In [256]:
from z3 import *

variables = [Int(name) for name in variable_names]

solver = Solver()

solver.add(Distinct(variables))

for var in variables:
    solver.add(Or([var == val for val in allowed_integers]))

variable_map = {str(var): var for var in variables}

for equation in equations:
    lhs, rhs = equation.split('=')
    terms = lhs.split('+')
    sum_terms = 0
    for term in terms:
        if term in variable_names:
            sum_terms += variable_map[term]
        else:
            sum_terms += int(term)
    solver.add(sum_terms == int(rhs.strip()))

def find_all_solutions(solver, variables):
    solutions = []
    while solver.check() == sat:
        model = solver.model()
        solution = {str(var): model[var].as_long() for var in variables}
        solutions.append(solution)
        solver.add(Or([var != model[var] for var in variables]))
    return solutions

all_solutions = find_all_solutions(solver, variables)

# Exist constraints

The second type of constraint specifies that for certain nodes, there must exist at least one non-self-intersecting path where the sum of the edge weights equals a given target sum. There can be multiple target sums for a single node.

This outlines the process for verifying the existence of non-self-intersecting paths in a graph, where the sum of the weights of the edges on the path must equal specified target values. This verification is performed using Depth First Search (DFS) for each possible solution generated from the sum constraints step.

In [257]:
def find_paths(graph: Graph, node, target_sum, visited):
    if target_sum == 0:
        return True
    if node in visited or target_sum < 0:
        return False
    visited.add(node)
    for neighbor in graph.neighbors(node):
        weight = graph.get_edge(node, neighbor)
        if weight > 0 and neighbor not in visited:
            if find_paths(graph, neighbor, target_sum - weight, visited):
                return True
    return False

for sol in all_solutions:
    graph = Graph(num_vertices) 

    for u, v, weight in edges_with_weights:
        graph.add_edge(u, v, sol.get(weight, weight))

    if all(find_paths(graph, node, target, set()) for node, targets in node_path_constraints.items() for target in targets):
        break

# Solution

This script calculates the shortest path in a weighted graph using Dijkstra's algorithm and then converts the path to a secret message. The vertices along the path are transformed into letters (1 = A, 2 = B, etc.).

The dijkstra_shortest_path function computes the shortest path using Dijkstra's algorithm.

The path is then converted to letters by mapping vertex numbers to their corresponding alphabetical characters (e.g., 1 -> A, 2 -> B).

Finally, the secret message is printed.

In [258]:
import heapq

def dijkstra_shortest_path(graph: Graph, start_vertex, end_vertex):
    distances = [float('inf')] * graph.vertices
    distances[start_vertex] = 0

    predecessors = [-1] * graph.vertices
    
    priority_queue = [(0, start_vertex)]
    
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)
        
        if current_distance > distances[current_vertex]:
            continue
        
        for neighbor in graph.neighbors(current_vertex):
            weight = graph.get_edge(current_vertex, neighbor)
            distance = current_distance + weight
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_vertex
                heapq.heappush(priority_queue, (distance, neighbor))
    
    path = []
    current = end_vertex
    while current != -1:
        path.append(graph.get_edge(current, predecessors[current]))
        current = predecessors[current]
    path.pop()
    path.reverse()
    
    return distances[end_vertex], path

distance, path = dijkstra_shortest_path(graph, 4, 17)
print(f"Shortest distance from vertex {4} to {17}: {distance}")
print(f"Path: {path}")
message = "".join(chr(p - 1 + ord('A')) for p in path)
print(f"Secret message: {message}")

Shortest distance from vertex 4 to 17: 55
Path: [12, 9, 14, 11, 5, 4]
Secret message: LINKED
